In [ ]:
import pandas as pd

# Opening datasets
df1 = pd.read_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/Data Science Projects/Database/Raw/CTIN1/elbw_usg_2012-2017.csv', index_col=0)
df2 = pd.read_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/Data Science Projects/Database/Raw/CTIN1/usg_2018-2021.csv', index_col=0)

df_ivh = pd.concat([df1, df2], axis=0)
df_ivh['date'] = df_ivh['Data_1'].fillna(df_ivh['Data_2'])
df_ivh = df_ivh.drop(['Data_1', 'Data_2'], axis=1)
df_ivh['Hemorragia'] = df_ivh['Hemorragia'].fillna('zero')

df1 = pd.read_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/Data Science Projects/Database/Raw/CTIN1/elbw_labs_2012-2017.csv')
df2 = pd.read_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/Data Science Projects/Database/Raw/CTIN1/labs_2018-2021.csv', sep=';')

df_labs = pd.concat([df1, df2], axis=0)

df1 = pd.read_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/Data Science Projects/Database/Raw/CTIN1/elbw_obitos_2012-2017.csv', index_col=0)
df2 = pd.read_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/Data Science Projects/Database/Raw/CTIN1/obitos_2018-2021.csv', index_col=0)

df_obitos = pd.concat([df1, df2], axis=0)
df_obitos = df_obitos.drop('Nome', axis=1)

df_char = pd.read_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/Data Science Projects/Database/Raw/CTIN1/uti_final.csv', sep=';')
df_char = df_char.drop(['name', 'nome_mae', 'nome_original', 'RNT', 'RNPT', 'data_dum', 'ig_dum', 'ig_dum_antes_usg', 'ig_dum_2', 'ig_usg', 'ig_usg_2', 'ig_usg_3', 'ig_usg_4'], axis=1)

df1 = pd.read_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/Data Science Projects/Database/Raw/CTIN1/elbw_echo_2012-2017.csv', index_col=0)
df2 = pd.read_csv('/Users/felipematsushita/Library/CloudStorage/GoogleDrive-fymatsushita@gmail.com/My Drive/Data Science Projects/Database/Raw/CTIN1/echo_2018-2021.csv', sep=';', index_col=0)

df_echo = pd.concat([df1, df2], axis=0)

# Cleaning IVH dataset
import re
def ivh_grade(grade_string):
    if bool(re.match(r'I\s', grade_string)) == True:
        return 1
    elif bool(re.match(r'II\s', grade_string)) == True:
        return 2
    elif bool(re.match(r'III\s', grade_string)) == True:
        return 3
    elif bool(re.match(r'IV\s', grade_string)) == True:
        return 4

df_ivh['ivh_grade'] = df_ivh['Hemorragia'].apply(ivh_grade).fillna(0)
df_ivh = df_ivh.drop('Hemorragia', axis=1)

# Only BW < 1500g + join deaths
df_char = df_char[df_char['Peso'] <= 1500]

def correct_ga(string):
    new_string = str(string).split(' ')
    if len(new_string) > 1:
        decimal = float(new_string[1][0])/7
        return str(round(float(new_string[0]) + decimal,2))
    else:
        return new_string[0]

df_char['GA'] = df_char['IG'].apply(correct_ga)
df_char = df_char.drop('IG', axis=1)
df_char['gemelar'] = df_char['gemelar'].fillna('single')
df_char = df_char.rename(columns={'dnasc_certo':'birthdate', 'Peso':'BW', 'sexo':'sex', 'gemelar':'twin'})

df_char = pd.merge(df_char, df_obitos, how='left', on='rghc')

def death(string):
    if string == '' or pd.isnull(string):
        return 'Survived'
    else:
        return 'Death'

df_char['outcome'] = df_char['data_obito'].apply(death)

# Cleaning ECHO dataset
df_echo = df_echo[df_echo['RGHC'].isin(df_char['rghc'])]
df_echo['date'] = df_echo['Data_1'].fillna(df_echo['Data_2'])

import re
def get_pda(string):
    if bool(re.match(r"\d(,|\.)\d$", str(string))) == True:
        return str(string)
    elif bool(re.match(r"\d\s$", str(string))) == True:
        return str(string)
    else:
        return None

df_echo['PDA'] = df_echo['PCA'].apply(get_pda)
df_echo['PDA_2'] = df_echo['PCA_2'].apply(get_pda)
df_echo['PDA'] = df_echo['PDA'].fillna(df_echo['PDA_2'])

df_echo['AO'] = df_echo['Ao'].fillna(df_echo['Ao_2']).fillna(df_echo['Ao_3'])
df_echo['LA'] = df_echo['AE'].fillna(df_echo['AE_2']).fillna(df_echo['AE_3'])

import numpy as np
df_echo['LA/AO'] = np.round(df_echo['LA'].str.replace(',','.').astype(float)/df_echo['AO'].str.replace(',','.').astype(float),2)

df_echo = df_echo.drop(['Data_1', 'Data_2', 'PCA', 'PCA_2', 'Ao', 'Ao_2', 'Ao_3', 'AE', 'AE_2', 'AE_3', 'FE', 'CIV', 'VTI', 'PSAP', 'VE', 'PDA_2', 'AO', 'LA'], axis=1)
df_echo['PDA'] = df_echo['PDA'].str.replace(',','.').astype(float).fillna(0)

df_echo = pd.merge(df_echo, df_char[['rghc', 'birthdate']], how='left', left_on='RGHC', right_on='rghc')

df_echo['date'] = pd.to_datetime(df_echo['date'], dayfirst=True, errors='coerce')
df_echo['birthdate'] = pd.to_datetime(df_echo['birthdate'], dayfirst=True, errors='coerce')

df_echo['data'] = (df_echo['date'] - df_echo['birthdate']).dt.days
df_echo = df_echo.dropna(subset=['data'])
df_echo = df_echo[df_echo['data'] < 15] # Only ECHO in the first 14 days of life

# Get worse IVH and worse PDA and join
df_echo_worse = df_echo.groupby('RGHC')['PDA'].max().reset_index()
df_LAAO_worse = df_echo.groupby('RGHC')['LA/AO'].max().reset_index()
df_ivh_worse = df_ivh.groupby('RGHC')['ivh_grade'].max().reset_index()

df_char = pd.merge(df_char, df_echo_worse, how='left', left_on='rghc', right_on='RGHC')
df_char = pd.merge(df_char, df_LAAO_worse, how='left', left_on='rghc', right_on='RGHC')
df_char = pd.merge(df_char, df_ivh_worse, how='left', left_on='rghc', right_on='RGHC')

df_char = df_char.drop(['RGHC_x', 'RGHC_y'], axis=1)

# Labs first 3 days of life
df_labs = df_labs[df_labs['RGHC'].isin(df_char['rghc'])]
df_lab = df_labs.drop('nome', axis=1)
df_lab['Date'] = pd.to_datetime(df_lab['Date'], dayfirst=True)
df_lab['DNASC'] = pd.to_datetime(df_lab['DNASC'], dayfirst=True)

df_dnasc = df_lab[['RGHC', 'DNASC']]
df_dnasc = df_dnasc.groupby('RGHC')['DNASC'].max().reset_index()

df_lab = pd.merge(df_lab, df_dnasc, how='left', on='RGHC')
df_lab = df_lab.drop('DNASC_x', axis=1)
df_lab['time'] = (df_lab['Date'] - df_lab['DNASC_y']).dt.days
df_lab = df_lab[df_lab['time'] < 4]

df_lab_select = df_lab[['RGHC', 'pH', 'ph_v', 'pCO2', 'pco2_v', 'ctCO2', 'BIC', 'bic_v', 'hco3bic', 'BE', 'be_v', 'Glic', 'glicose_v', 'glicgaso', 'Lactato', 'lactato_v', 'lactatogaso', 'Hb', 'Hb2', 'Ht', 'Ht2', 'VCM', 'HCM', 'CHCM', 'RDWCV', 'RDWSD', 'Leuco', 'leuco_absoluto', 'Neutro', 'ProMielo', 'promielo_absoluto', 'Mielo', 'mielo_absoluto', 'Metamielo', 'metamielo_absoluto', 'Bast', 'bast_absoluto', 'Segmentados', 'seg_absoluto', 'Eosino', 'eosino_absoluto', 'Baso', 'baso_absoluto', 'Linfo', 'linf_absoluto', 'Mono', 'mono_absoluto', 'blastos', 'Plaq', 'plaq_2', 'VPM', 'PCR', 'Na', 'K', 'Cl', 'P', 'Mg', 'CaT', 'Cai', 'cai_v', 'Ur', 'Creat', 'BT', 'BD', 'BI', 'Tipagem', 'CD', 'HMCperi', 'HMC_cateter', 'HMC_cateter_2', 'troponina']]

df_lab_select['pH'] = df_lab_select['pH'].str.replace(',','.').astype(float)
df_lab_select['ph_v'] = df_lab_select['ph_v'].str.replace(',','.').astype(float)
df_lab_select['pH'] = df_lab_select['pH'].fillna(df_lab_select['ph_v'])

df_lab_select['pCO2']= df_lab_select['pCO2'].fillna(df_lab_select['pco2_v'])
df_lab_select['pCO2'] = df_lab_select['pCO2'].str.replace(',','.').astype(float)

df_lab_select['ctCO2'] = df_lab_select['ctCO2'].str.replace(',','.').astype(float)

df_lab_select['BIC'] = df_lab_select['BIC'].fillna(df_lab_select['bic_v']).fillna(df_lab_select['hco3bic'])
df_lab_select['BIC'] = df_lab_select['BIC'].str.replace(',','.').astype(float)

df_lab_select['BE'] = df_lab_select['BE'].fillna(df_lab_select['be_v'])
df_lab_select['BE'] = df_lab_select['BE'].str.replace(',','.').astype(float)

df_lab_select['Glic'] = df_lab_select['Glic'].fillna(df_lab_select['glicose_v']).fillna(df_lab_select['glicgaso'])

df_lab_select['lactato_v'] = df_lab_select['lactato_v'].str.replace(',','.').astype(float)
df_lab_select['lactatogaso'] = df_lab_select['lactatogaso'].str.replace(',','.').astype(float)
df_lab_select['Lactato'] = df_lab_select['Lactato'].fillna(df_lab_select['lactato_v']).fillna(df_lab_select['lactatogaso'])

df_lab_select['Hb'] = df_lab_select['Hb'].fillna(df_lab_select['Hb2']).str.replace(',','.').astype(float)
df_lab_select['Ht'] = df_lab_select['Ht'].fillna(df_lab_select['Ht2']).str.replace(',','.').astype(float)

df_lab_select['Plaq'] = df_lab_select['Plaq'].fillna(df_lab_select['plaq_2'])

df_lab_select['Cai'] = df_lab_select['Cai'].fillna(df_lab_select['cai_v']).str.replace(',','.').astype(float)

df_lab_select['VCM'] = df_lab_select['VCM'].str.replace(',','.').astype(float)
df_lab_select['HCM'] = df_lab_select['HCM'].str.replace(',','.').astype(float)
df_lab_select['CHCM'] = df_lab_select['CHCM'].str.replace(',','.').astype(float)
df_lab_select['RDWCV'] = df_lab_select['RDWCV'].str.replace(',','.').astype(float)
df_lab_select['RDWSD'] = df_lab_select['RDWSD'].str.replace(',','.').astype(float)
df_lab_select['Leuco'] = df_lab_select['Leuco'].str.replace(',','.').astype(float)
df_lab_select['leuco_absoluto'] = df_lab_select['leuco_absoluto'].str.replace(',','.').astype(float)
df_lab_select['Neutro'] = df_lab_select['Neutro'].str.replace(',','.').astype(float)
df_lab_select['ProMielo'] = df_lab_select['ProMielo'].str.replace(',','.').astype(float)
df_lab_select['promielo_absoluto'] = df_lab_select['promielo_absoluto'].str.replace(',','.').astype(float)
df_lab_select['Mielo'] = df_lab_select['Mielo'].str.replace(',','.').astype(float)
df_lab_select['mielo_absoluto'] = df_lab_select['mielo_absoluto'].str.replace(',','.').astype(float)
df_lab_select['Metamielo'] = df_lab_select['Metamielo'].str.replace(',','.').astype(float)
df_lab_select['metamielo_absoluto'] = df_lab_select['metamielo_absoluto'].str.replace(',','.').astype(float)
df_lab_select['Bast'] = df_lab_select['Bast'].str.replace(',','.').astype(float)
df_lab_select['bast_absoluto'] = df_lab_select['bast_absoluto'].str.replace(',','.').astype(float)
df_lab_select['Segmentados'] = df_lab_select['Segmentados'].str.replace(',','.').astype(float)
df_lab_select['seg_absoluto'] = df_lab_select['seg_absoluto'].str.replace(',','.').astype(float)
df_lab_select['Eosino'] = df_lab_select['Eosino'].str.replace(',','.').astype(float)
df_lab_select['eosino_absoluto'] = df_lab_select['eosino_absoluto'].str.replace(',','.').astype(float)
df_lab_select['Baso'] = df_lab_select['Baso'].str.replace(',','.').astype(float)
df_lab_select['baso_absoluto'] = df_lab_select['baso_absoluto'].str.replace(',','.').astype(float)
df_lab_select['Linfo'] = df_lab_select['Linfo'].str.replace(',','.').astype(float)
df_lab_select['linf_absoluto'] = df_lab_select['linf_absoluto'].str.replace(',','.').astype(float)
df_lab_select['Mono'] = df_lab_select['Mono'].str.replace(',','.').astype(float)
df_lab_select['mono_absoluto'] = df_lab_select['mono_absoluto'].str.replace(',','.').astype(float)
df_lab_select['blastos'] = df_lab_select['blastos'].str.replace(',','.').astype(float)
df_lab_select['VPM'] = df_lab_select['VPM'].str.replace(',','.').astype(float)
df_lab_select['PCR'] = df_lab_select['PCR'].str.replace(',','.').astype(float)
df_lab_select['K'] = df_lab_select['K'].str.replace(',','.').astype(float)
df_lab_select['Cl'] = df_lab_select['Cl'].str.replace(',','.').astype(float)
df_lab_select['P'] = df_lab_select['P'].str.replace(',','.').astype(float)
df_lab_select['Mg'] = df_lab_select['Mg'].str.replace(',','.').astype(float)
df_lab_select['CaT'] = df_lab_select['CaT'].str.replace(',','.').astype(float)
df_lab_select['Creat'] = df_lab_select['Creat'].str.replace(',','.').astype(float)
df_lab_select['BT'] = df_lab_select['BT'].str.replace(',','.').astype(float)
df_lab_select['BD'] = df_lab_select['BD'].str.replace(',','.').astype(float)
df_lab_select['BI'] = df_lab_select['BI'].str.replace(',','.').astype(float)
df_lab_select['troponina'] = df_lab_select['troponina'].str.replace(',','.').astype(float)

df_lab_select = df_lab_select.drop(['ph_v', 'pco2_v', 'bic_v', 'hco3bic', 'be_v', 'glicose_v', 'glicgaso', 'lactato_v', 'lactatogaso', 'Hb2', 'Ht2', 'plaq_2', 'cai_v', 'HMCperi', 'HMC_cateter', 'HMC_cateter_2', 'Tipagem', 'CD'], axis=1)

df_lab_avg = df_lab_select.groupby('RGHC').mean().reset_index()
df_lab_avg = df_lab_avg.add_suffix('_avg')

df_lab_min = df_lab_select.groupby('RGHC').min().reset_index()
df_lab_min = df_lab_min.add_suffix('_min')

df_lab_max = df_lab_select.groupby('RGHC').max().reset_index()
df_lab_max = df_lab_max.add_suffix('_max')

df_select = df_lab.groupby('RGHC')['DNASC_y'].max().reset_index()
df_final = pd.merge(df_char, df_select, how='left', left_on='rghc', right_on='RGHC')
df_final = pd.merge(df_final, df_lab_avg, how='left', left_on='rghc', right_on='RGHC_avg')
df_final = pd.merge(df_final, df_lab_min, how='left', left_on='rghc', right_on='RGHC_min')
df_final = pd.merge(df_final, df_lab_max, how='left', left_on='rghc', right_on='RGHC_max')
df_final = np.round(df_final, 2)
df_final = df_final.drop(['birthdate', 'RGHC_avg', 'RGHC_min', 'RGHC_max'], axis=1)

df_final = df_final.drop(['leuco_absoluto_avg', 'leuco_absoluto_max', 'leuco_absoluto_min', 'promielo_absoluto_avg', 'promielo_absoluto_max', 'promielo_absoluto_min', 'mielo_absoluto_avg', 'mielo_absoluto_max', 'mielo_absoluto_min', 'metamielo_absoluto_avg', 'metamielo_absoluto_max', 'metamielo_absoluto_min', 'seg_absoluto_avg', 'seg_absoluto_max', 'seg_absoluto_min', 'bast_absoluto_avg', 'bast_absoluto_max', 'bast_absoluto_min', 'eosino_absoluto_avg', 'eosino_absoluto_max', 'eosino_absoluto_min', 'baso_absoluto_avg', 'baso_absoluto_max', 'baso_absoluto_min',  'linf_absoluto_avg', 'linf_absoluto_max', 'linf_absoluto_min', 'mono_absoluto_avg', 'mono_absoluto_max', 'mono_absoluto_min', 'blastos_avg', 'blastos_max', 'blastos_min', 'troponina_avg', 'troponina_min', 'troponina_max', 'data_obito'], axis=1)

In [90]:
df_final.to_csv('dataset.csv')